In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy
from control import lqr, dare
from single_photons.estimators.kalman import KalmanFilter
import single_photons.utils.constants as ct
from single_photons.utils.parameters import *
from single_photons.utils.metrics import *
from single_photons.environment import Particle


KeyboardInterrupt



In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams.update({
    "text.usetex": True,
})
plt.rcParams['figure.dpi'] = 150
C = ['#2E86C1', '#85C1E9', '#1B4F72']

In [ ]:
omega = 2*np.pi*104e3
p = 9.2e-7
radius = 71.5e-9
wavelength = 1.064e-6
power = 300e-3
waist = 3.7352e-6 #from scattered power in Magrini paper. resulting theoretical omega is 56% the experimental one
eta_detection = 0.178
delta_t = 1e-9
control_step = int(32e-9/delta_t) 
fs = 1/(control_step*delta_t)
gamma, ba_force, std_detection, std_z = compute_parameters_simulation(power, wavelength, waist, omega,
                                                                             radius, p, fs, eta_detection)
coupling = (1/(4*np.pi))*(ba_force**2)
env = Particle(omega, gamma, coupling, eta_detection=eta_detection)
variance_process = env.thermal_force_std**2 + env.backaction_std**2
std_detection = std_detection/env.zp_x
period = 2*np.pi/omega
t = np.arange(0, 100 * period, delta_t)
N = t.shape[0]
g_fb_ratio = 0.5

In [ ]:
Q = np.array([[0, 0], [0, variance_process]])*control_step*delta_t/2
R = np.array([[np.power(std_detection,2)]])

In [ ]:
g_fb = g_fb_ratio*omega
Ad = scipy.linalg.expm(env.A *control_step*delta_t)
Bd = env.B * delta_t * control_step
cost_states = np.array([[omega/2, 0],
                        [0, omega/2]])
(G_lqr, S, E) = lqr(env.A, env.B, cost_states, omega/(g_fb**2))
X, L, G = dare(Ad, Bd, cost_states, omega/(g_fb**2))
G_lqr, G

In [ ]:
x0 = std_detection
P0 = std_detection**2*np.matrix(np.eye(2))
estimation = np.matrix([[x0*np.random.normal()], [x0*np.random.normal()]])
states = np.array([[x0*np.random.normal()], [x0*np.random.normal()]])
new_states = np.zeros((N, 2))
measured_states = np.zeros((N))
estimated_states = np.zeros((N, 2))
estimated_states[0, :] = estimation.reshape((2))
estimation = estimation.reshape((2, 1))
control = np.array([[0]])
controls = []
kalman = KalmanFilter(estimation, P0, Ad, Bd, env.C, Q, R)
for i in tqdm(range(t.shape[0])):
    new_states[i, :] = states[:, 0]
    if not i % control_step:
        measured_states[i] = states[0,0] + std_detection * np.random.normal()
        kalman.propagate_dynamics(control)
        kalman.compute_aposteriori(measured_states[i])
        estimated_states[i, :] = kalman.estimates_aposteriori[int(i/control_step)][:, 0].reshape((2))
        estimation = estimated_states[i, :].reshape((2, 1))
        control = -G @ estimation
    else:
        measured_states[i] = measured_states[i-1]
        estimated_states[i, :] = estimated_states[i-1,:]
    controls.append(float(control))
    states = env.step(states, control=control, delta_t=delta_t)

In [ ]:
#plt.plot(1e3*t[::control_step],env.zp_x*1e9*measured_states[::control_step], color = C[0], alpha = 0.2)
plt.plot(1e3*t[::control_step],env.zp_x*1e9*new_states[::control_step,0], color = C[1], alpha = 0.6)
plt.plot(1e3*t[::control_step],env.zp_x*1e9*estimated_states[::control_step,0], color = C[2], alpha = 0.4)
plt.ylabel(r'$\langle z \rangle$ (nm)')
plt.xlabel(r'time (ms)')
plt.grid(alpha = 0.4)

In [ ]:
step=10
phonons = compute_phonons(estimated_states, np.array(kalman.error_covariance_aposteriori), control_step, step=step)
plt.plot(t[::control_step][::step][2:]/1e-6, phonons)
plt.ylabel(r'$\langle \bar{n} \rangle$')
plt.xlabel(r'$t[\mu s]$')
plt.yscale('log')
plt.grid()
plt.show()
print(min(phonons))

In [ ]:
cov_mat = kalman.error_covariance_aposteriori[-1]
z_std = env.zp_x*np.sqrt(cov_mat[0,0])
p_std = env.zp_p*np.sqrt(cov_mat[1,1])


In [ ]:
fig = plt.Figure()
plt.title('Position')
plt.plot(t, measured_states)
plt.plot(t[100:], estimated_states[100:,0])
plt.plot(t[100:], new_states[100:,0])
plt.legend(['Measured', 'Estimation', 'Simulation'], loc='upper right')
plt.xlabel(r't(s)')
plt.ylabel(r'$\langle z\rangle/z_{zp}$')
plt.grid()
plt.show()

In [ ]:
fig = plt.Figure()
plt.title('Velocity')
plt.plot(t[:], estimated_states[:,1])
plt.plot(t[:], new_states[:,1])
plt.grid()
plt.show()

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
df = pd.DataFrame()
df['z'] = butter_bandpass_filter(estimated_states[:,0][::30], 22e3, 220e3, 1/(30e-9), order=2)[500:]
df['p'] = butter_bandpass_filter(estimated_states[:,1][::30], 22e3, 220e3, 1/(30e-9), order=2)[500:]

In [ ]:
plt.plot(df['z'][100:])
plt.plot(df['p'][100:])
df['z'].values[0], df['p'].values[0]

In [ ]:
sns.set_theme(style="white")
g = sns.JointGrid(data=df, x="z", y="p", space=0, xlim=[-3,3], ylim=[-3,3])
g.plot_joint(sns.lineplot, sort = False)#clip=((2200, 6800), (10, 25)),
             #thresh=0, levels=100, cmap="rocket")
g.plot_marginals(sns.kdeplot, alpha=1, fill=True)
g.set_axis_labels(r'$z/z_{zp}$',r'$p/p_{zp}$')

In [30]:
(1/np.sqrt(delta_t))*1.76*np.sqrt(4e-28)/env.zp_x

587.1995258653128

In [17]:
1e6*compute_scattered_power(power,waist,wavelength)/22.4

5.336160333389444

In [20]:
compute_omega(power, waist)/(2*np.pi)/104000

0.5635880979043871

In [7]:
print(env.C)

[[1. 0.]]
